## Scale invariant feature transform (SIFT) 


This tutorial mainly follows:


1. The original paper (a recommended read): [{Distinctive Image Featuresfrom Scale-Invariant Keypoints}](https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf)
2. Another tutorial at [{AI Shack | SIFT: Theory and Practice}](https://aishack.in/tutorials/sift-scale-invariant-feature-transform-introduction/) with adapting to new data and other considrations.

### Problem definition

* Features or **key-points** of an image are corners which are unique in the image.
* Harris is a corner detector, we have discussed Previously. 
* Corner detectors are invariant for translation, illumination and rotation. **But it is variant for scaling**. 

### Example

Next figure shows two different scales of same image. 

<span style="display:block;text-align:center"><img style="width:75%"  src="media/sift_scale_invariant.jpg"></span>

* In smaller image,  it's easy to detect that there is a corner, but what about same image in the large scale. It will be difficult to detect that corner so this feature point will not be recognized for all scales. 
* So size of the window will effect the detection of corners. Large corners needs large windows and smaller corners needs smaller windows. 

Scale invariant feature descriptor (SIFT) is not a new way to find key-points or corners that is invariant to scale. But it is a descriptor of detected corners of different image scales or image pyramids. 

### A) Image pyramids and scale-spaces

Image pyramids or image scale space is the proposed method to handle images in different scales. We have different scale-spaces 

* Gaussian scale space (Gaussian pyramid)
* Laplacian of gaussian (LOG) scale space 
* Difference of gaussian (DOG) scale space 

The basic idea to build scale space is shown in the following figure 

**Gaussian Pyramid**

<span style="display:block;text-align:center"><img style="width:55%"  src="media/Image_pyramid.png"></span>


[source](https://en.wikipedia.org/wiki/Pyramid_(image_processing))

**LOG Pyramid**

<span style="display:block;text-align:center"><img style="width:85%"  src="media/LOG.png"></span>


In SIFT we usually prefer DOG scale space which is an approximate of LOG and simpler in calculation. 

### SIFT scale space (DOG)

In SIFT Pyramid we have 

* Octaves: different levels of image resolutions (pyramids levels)
* Scales: different scales of window in each octave level (different $\sigma$ of gaussian window)

![](media/sift_dog.jpg)

#### Practical details of constructing the scale space in SIFT

The following practices suggested by the SIFT author ([{Distinctive Image Featuresfrom Scale-Invariant Keypoints}](https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf) p. 7):


1. Overall, we consider four octaves (resolution levels, or image shapes). Each octave shape is $\times 2$ subsampling of its previous octave. So if first octave resolution is (64,64), the next resolutions are (32,32), (16,16), and (8,8).
1. For each octave, we consider five different blurred images ($\sigma, k \sigma, k^2 \sigma, k^3 \sigma, k^4 \sigma$); and $k=\sqrt{2}$. Hence, the squence of Gaussian blurring becomes ($\sigma,\sqrt{2} \sigma, 2 \sigma, 2\sqrt{2} \sigma, 4 \sigma$).
1. We start the first octave on a $\times 2$ upsampled from the input image. This allows extracting features on a ~~larger~~ smaller scale (because the relative size of Harris operator to the image decreases). So if the input image is (128,128), the four octaves will have the following resolutions: (256,256), (128,128), (64,64), and (32,32).
1. For the $2^{nd}-$, $3^{rd}-$, and $4^{th}-$octaves, we $\times 2$ subsample the image from the previous octave that has twice $\sigma$ of the initial image (which mean the third image). 
1. Finally, from each octave, we obtain 4 DoG images.
1. $\sigma = 1.6$ is chosen by the author for stable results (repeatability).

##### Let's declare the previous practical considerations before starting

In [1]:
from scipy.signal import convolve2d
from cvutils import gaussian_kernel2d
from skimage.transform import rescale

# The following are suggested by SIFT author
N_OCTAVES = 4 
N_IMAGES_PER_OCTAVE = 5 
SIGMA = lambda s: [ (2.0**i)*s for i in range(5) ] # (s, √2s , 2s, 2√2 s , 4s )
SIGMA_SIFT = SIGMA(1.6) #
KERNELS_SIFT = [ gaussian_kernel2d(std = s, kernlen = 4 * int(s+0.5) + 1) for s in SIGMA_SIFT ]

ModuleNotFoundError: No module named 'scipy'

In [2]:
def image_dog( img ):
    octaves = []
    dog = []
    for i in range(4):
        base = rescale( img, 2, anti_aliasing=False) if i == 0 else rescale( octaves[i-1][2], 0.5 , anti_aliasing = False ) 
        octaves.append([ convolve2d( base , kernel , 'same') for kernel in KERNELS_SIFT ])
        dog.append([ s2 - s1 for (s1,s2) in zip( octaves[i][:-1], octaves[i][1:])]) # ;)
    return dog

### Key-point (corner) scale localization

For each key-point (corner) we need to find its best scale which have maximum value (cornerness measure). It is achieved by comparing same corner with its neighbors of above and lower scales and select scale with maximum value. For same iamge, it is not necessary for its corners to be localized at same scale.

![](media/sift_local_extrema.jpg)

### Extract SIFT feature descriptor

![](media/sift-fingerprint.jpg)

[source](http://aishack.in/tutorials/sift-scale-invariant-feature-transform-features/)

After localization of a key-point in our scale space. We can get its SIFT descriptor as follow

* Extract a $$16 \times 16$$ window centered by this point.
* Get gradient magnitude and multiply it by a $$16 \times 16$$ gaussian window of $$\sigma =1.5$$
* Get gradient angle direction. 
* Adjusting orientation (To be rotation invariant):
    * Get the gradient angle of the window and Quantize them to 36 values (0, 10, 20, ..., 360)
    * Locate dominant corner direction which is most probable angle (angle with max value in 36 bit angle histogram)
    * subtract dominant direction from gradient angle.

![](media/siftOriented.png)

* Divide this $$16 \times 16$$ patch to sixteen $$4 \times 4$$ blocks
* For each block get magnitude weighted angle histogram and normalize it (divide by total gradient magnitudes). 
 
 angles (quantized to 8 angles [0, 45, 90, ... , 360]) based on its relevant gradient magnitude i.e (histogram of angle 0 = sum(all magnitudes with angle 0))

 ![](media/histangles.png)



* SIFT feature descriptor will be a vector of 128 element (16 blocks $$\times$$ 8 values from each block)

## Feature matching

The basic idea of feature matching is to calculate the sum square difference between two different feature descriptors (SSD). So feature will be matched with another with minimum SSD value. 

$$
SSD = \sum (v_1 - v_2)^2
$$

where $$v_1$$ and $$v_2$$ are two feature descriptors.

![](media/matcher_result1.jpg)


### Brute-Force matcher

In brute-force matcher we have to match descriptor of all features in an image to descriptors of all features in another image. It is extremely expensive as we know any brute-force algorithm will guarantee getting a solution, but doesn't guarantee getting optimal solution.

### RANSAC 

Random sample consensus is an iterative method for estimation of parameters of a mathematical model. We will model the transformation of points in source image to destination one, and try to find an estimate of model parameters. The basic idea of RANSAC algorithm is shown in the following flow chart. 

<!-- ![](media/ransac.png) -->

RANSAC is a robust feature matcher. For example we can model the difference between two images to a set of transformations and run RANSAC to find best model that maximize correct matching.